In [1]:
import json
import os
import random
import time

In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get IHOP's Data

In [23]:
scraper = cloudscraper.create_scraper()

In [30]:
def get_coords(store_url: str) -> Point:
    ##store_url = 'https://restaurants.ihop.com/en-us/nc/garner/breakfast-1450-us-hwy-70-w-4434'
    href = "/".join(store_url.split("/")[4:])
    store_json_url = os.path.join(
        "https://api.wordlift.io/data/https/restaurants.ihop.com/en-us", href
    )
    r = scraper.get(store_json_url)
    if r.status_code != 200 or not r.json():
        print(store_json_url)
        return
    store_json = r.json()[0]
    lat = float(store_json["geo"]["latitude"])
    lon = float(store_json["geo"]["longitude"])
    return Point(lon, lat)

In [31]:
url_root = "https://restaurants.ihop.com/en-us/"

# Parse all locations in a state
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    state_code = states_df.iloc[i]["STUSPS"].lower()
    state_url = f"{url_root}{state_code}/"
    state_store_list = []

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    r_state = scraper.get(state_url)

    if r_state.status_code != 200:
        print(state_code)
        continue

    soup_state = BeautifulSoup(r_state.text, "html.parser")
    location_lis = soup_state.find_all("li", {"class": "map-list-item-wrap is-single"})
    city_as = [location_li.find("a") for location_li in location_lis]

    for city_a in tqdm(city_as, desc=f"Parsing Locations in {state_code}"):
        city_url = city_a.attrs["href"]
        r_city = scraper.get(city_url)
        soup_city = BeautifulSoup(r_city.text, "html.parser")
        store_divs = soup_city.find_all("div", {"class": "map-list-item-header"})
        store_as = [store_div.find("a") for store_div in store_divs]

        for store_a in store_as:
            try:
                store_url = store_a.attrs["href"]
                point = get_coords(store_url)
            except Exception as e:
                print(store_url, e)
                continue
            store_dict = {"STUSPS": state_code, "geometry": point}
            print(point)
            state_store_list.append(store_dict)
            time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        print(len(state_store_list), "stores")
        state_IHOP_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        state_IHOP_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing Locations in ms:   0%|          | 0/12 [00:00<?, ?it/s]

https://api.wordlift.io/data/https/restaurants.ihop.com/en-us/ms/biloxi/breakfast-2680-beach-blvd-451
None
https://api.wordlift.io/data/https/restaurants.ihop.com/en-us/ms/diberville/breakfast-11475-cinema-way-dr-2106
None
https://api.wordlift.io/data/https/restaurants.ihop.com/en-us/ms/flowood/breakfast-5076-lakeland-drive-3646
None
https://api.wordlift.io/data/https/restaurants.ihop.com/en-us/ms/gulfport/breakfast-15130-turkey-creek-dr-4484
None


KeyboardInterrupt: 

In [41]:
ihop_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        ihop_state_gdf = gpd.read_file(constructed_path)
        ihop_state_gdfs.append(ihop_state_gdf)

In [42]:
ihop_state_gdf = gpd.GeoDataFrame(pd.concat(ihop_state_gdfs, ignore_index=True))

In [43]:
ihop_state_gdf = ihop_state_gdf.to_crs(9311)
ihop_state_gdf.to_file(f"data/ihops.gpkg")

## Get Population Data

In [44]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [45]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [46]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [47]:
states_with_population_df = states_with_population_df.dropna()

## Merge Data

In [54]:
ihop_state_counts_df = pd.DataFrame(
    ihop_state_gdf.groupby("STUSPS").size(), columns=["IHOPs"]
).reset_index()

In [55]:
ihop_state_counts_pop_df = states_with_population_df.merge(
    ihop_state_counts_df, on="STUSPS", how="left"
)
ihop_state_counts_pop_df = ihop_state_counts_pop_df.fillna(0)

In [56]:
ihop_state_counts_pop_df["per_1000"] = ihop_state_counts_pop_df["IHOPs"] / (
    ihop_state_counts_pop_df["POPULATION"] / 1000
)
ihop_state_counts_pop_df["per_10k"] = ihop_state_counts_pop_df["IHOPs"] / (
    ihop_state_counts_pop_df["POPULATION"] / 10_000
)
ihop_state_counts_pop_df["per_100k"] = ihop_state_counts_pop_df["IHOPs"] / (
    ihop_state_counts_pop_df["POPULATION"] / 100000
)
ihop_state_counts_pop_df["per_500k"] = ihop_state_counts_pop_df["IHOPs"] / (
    ihop_state_counts_pop_df["POPULATION"] / 500_000
)
ihop_state_counts_pop_df["per_1m"] = ihop_state_counts_pop_df["IHOPs"] / (
    ihop_state_counts_pop_df["POPULATION"] / 1_000_000
)
ihop_state_counts_pop_df["per_capita"] = (
    ihop_state_counts_pop_df["POPULATION"] / ihop_state_counts_pop_df["IHOPs"]
)

In [57]:
ihop_state_counts_pop_df = ihop_state_counts_pop_df.to_crs(9311)

In [58]:
ihop_state_counts_pop_df.to_file("data/IHOPs_per_state.gpkg")